# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import statistics
%matplotlib inline

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

run your solution on all test_images and make copies into the test_images directory).

In [2]:
## THIS IS THE CODE USED TO GENERATE THE IMAGES. IT HAS FUNCTION CALLS TO FUNCTIONS THAT HAVE SINCE BEEN 
## MODIFIED TO WORK WITH VIDEO PROCESSING PIPELINE. AS SUCH, THIS FUNCTION WILL NO LONGER SUCCESSFULLY
## BUILD AND OUTPUT

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import csv
%matplotlib inline

global MASK_Offset_BottomHorizon
global MASK_Offset_Horizon
global Video_Type

## CONSTANTS START
# Call type
# Gaussian
kernel_size = 5 #Result from homework lesson
# Canny Edge
low_threshold = 25
high_threshold = 75
# Hough
rho = 2
theta = np.pi/180
threshold = 15
min_line_length = 50
max_line_gap = 40
## CONSTANTS END

## INIT START
# Masking
MASK_Offset_Horizon_Correction = 50
MASK_Offset_BotLeft = 50
MASK_Offset_BotRight = 50
MASK_Offset_TopLeft = 25
MASK_Offset_TopRight = 25
ignore_mask_color = 255
# Image Info
image_info = 'test_images/whitecarLaneSwitch.jpg'
output_info = 'test_images/OUTPUT_whiteCarLaneSwitch.jpg'
## INIT END

## Base image info and Masking parameter setup
image = cv2.imread(image_info)
ysize = image.shape[0]
xsize = image.shape[1]
MASK_Offset_Horizon = int(ysize / 2 + MASK_Offset_Horizon_Correction)
MASK_Offset_BottomHorizon = ysize
print(MASK_Offset_BottomHorizon)

# print('This image is with dimesions:', image.shape) # Used for testing
## GRAYSCALE
## Utilize OpenCV for grayscale conversion
image_gray = grayscale(image)

## GAUSSIAN SMOOTHING
## Utilize OpenCV for gaussian smoothing
blur_gray = gaussian_blur(image_gray,kernel_size)

## CANNY EDGE
## Utilize OpenCV for Canny Edge Detection
edges = canny(blur_gray,low_threshold,high_threshold)

## MASKING
## Utilize numpy for init of mask of zero's for targetted image extraction
mask = np.zeros_like(edges)

## Create four sided polygon array for targetted image extraction
vertices = np.array([[(MASK_Offset_BotLeft,MASK_Offset_BottomHorizon),(xsize/2-MASK_Offset_TopLeft,MASK_Offset_Horizon),
                      (xsize/2+MASK_Offset_TopRight,MASK_Offset_Horizon),(xsize-MASK_Offset_BotRight, MASK_Offset_BottomHorizon)]],
                    dtype=np.int32)
cv2.fillPoly(mask,vertices,ignore_mask_color)
masked_edges = cv2.bitwise_and(edges,mask)


line_image = np.copy(image)*0

#hough_out = hough_extrap(masked_edges,rho,theta,threshold,min_line_length,max_line_gap,image)
hough_out = hough_lines(masked_edges,rho,theta,threshold,min_line_length,max_line_gap)

# hough_gray = grayscale(hough_out)
#fl = open('hough_output.csv', 'w')

#writer = csv.writer(fl)
#for values in hough_gray:
#   writer.writerow(values)

#fl.close() 

print(MASK_Offset_BottomHorizon)
weighted_out = weighted_img (hough_out,image)
plt.imshow(masked_edges)
cv2.imwrite(output_info,weighted_out)



540


NameError: name 'grayscale' is not defined

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [2]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [3]:
import math


def grayscale(img):
    """ NO CHANGES TO FUNCTION
    Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    #return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """NO CHANGES TO FUNCTION
    Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """NO CHANGES TO FUNCTION
    Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def create_lines(img, lines, Count, color=[0, 0, 255], thickness=10):
    """
   This function became the big meat of my pipeline.  It finds seperates the lines into left/right arrays, 
   extracts slope, intercept, and length of these lines, averages them, extrapolates them and plots them. Wow!
   
   !! Note !!
   The extrapolation can cause lines to pass beyond the masked scope prior to Hough-Transform
   
   ?? Masking can be applied again (?) after if needed ??
   
    """
    ## Globals
    global Right_Line_Filter
    global Left_Line_Filter
    global Right_Line_Info
    global Left_Line_Info
    
    ## Array Initialization
    left_lines_info = []
    right_lines_info = []
    left_lines =[]
    right_lines=[]
    right_ave = []
    left_ave = []
    
    ## Allowable slope initialization
    if Count <=5:    #If count is below 5, the slope bounds are wider to allow averaging to take place
        Slope_Bound = 0.2
    else:
        Slope_Bound = 0.15
    
    Right_Slope_Max = Right_Line_Filter[0]+Slope_Bound
    Right_Slope_Min = Right_Line_Filter[0]-Slope_Bound
    Left_Slope_Max = Left_Line_Filter[0]+Slope_Bound
    Left_Slope_Min = Left_Line_Filter[0]-Slope_Bound

    
    ## Loop through Hough_Transform Output, create line y=Mx+B information along with line length
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            line_length = math.hypot(y2 - y1, x2 - x1) #Python math function
            # With a slope greater then 0.5, this line will be on the right side of the road from the camera's perspective
            if (slope < Right_Slope_Max and slope > Right_Slope_Min):               
                line_input = (slope,intercept,line_length)
                right_lines_info.append(line_input)
            # With a slope less then 0.5, this line will be on the left side of the road from the camera's perspective
            elif (slope < Left_Slope_Max and slope > Left_Slope_Min):  
                line_input = (slope,intercept,line_length)
                left_lines_info.append(line_input)
    
    ##Call averaging functions - this averages all line segments determined to be left v right
    right_ave = side_average(right_lines_info)
    left_ave = side_average(left_lines_info)

    ## Call moving average function - this function is just a disappointment. Room for improvement
    Line_Averaging(right_ave,left_ave,Count)

    ##Extrapolate lines - Extend the averaged line to the boundaries (top and bottom)
    Right_Line_Info = (side_line_extrap(Right_Line_Filter))
    Left_Line_Info = (side_line_extrap(Left_Line_Filter))
    
    
    ##Add right and left lines onto the image
    cv2.line(img, (Right_Line_Info[0],Right_Line_Info[1]), (Right_Line_Info[2], Right_Line_Info[3]), color, thickness)
    
    cv2.line(img, (Left_Line_Info[0],Left_Line_Info[1]), (Left_Line_Info[2], Left_Line_Info[3]), color, thickness)


def side_average (line_info):
    """
    In this function, a 'sides' lines are averaged
    
    In this rudimentary averaging application, the hough_transform's output for 'line length'
    is utilized for a simple weighting
    
    """
    ## Initialize variables
    column = 2 #Column index 2 points to line lengths
    slope = 0 # Start value for summation
    intercept = 0 # Start value for summation
    ## Sum all line lengths for averaging info
    total_length = sum(row[column] for row in line_info) #Google search pointed to StackOverFlow
    ## http://stackoverflow.com/questions/23145684/sum-matrix-columns-in-python
    ## Loop to accumulate averaging slopes and intercepts
    ## Loop through all lines in line_info and apply a very simply averaging to slopes and intercepts
    for line in line_info:
        slope += line[0] *line[2]/total_length
        intercept += line[1] * line[2] / total_length
    
    return (slope,intercept)
        
def side_line_extrap(line_info):
    """
    Simple function to extrapolate a line segment utilizing the natural borders of the forced horizon
    and the bottom of the image
    """
    
    ## Initialization
    line_out = np.zeros(4)
    

    ## Find x coords for each two points as y points are simply the forced boundaries
    horizon_point_x = int((MASK_Offset_Horizon - line_info[1]) / line_info[0])
    bottom_point_x = int((MASK_Offset_BottomHorizon - line_info[1]) / line_info[0])
    #append_cheat = [horizon_point_x,MASK_Offset_Horizon,bottom_point_x,MASK_Offset_BottomHorizon] #To simplify append
    line_out = (horizon_point_x,MASK_Offset_Horizon,bottom_point_x,MASK_Offset_BottomHorizon)
    return line_out
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap,Count):
    """ NO CHANGES TO THIS FUNCTION
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    create_lines(line_img, lines,Count)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def Init_Mask(image):
    """
    This function creates the initial mask that is applied. Not much goes on here...
    
    """
    global Left_Line_Mask
    global Right_Line_Mask
    global MASK_Offset_BottomHorizon
    global MASK_Offset_Horizon
    
    MASK_Offset_Horizon_Correction = 50
    MASK_Offset_BotLeft = 50
    MASK_Offset_BotRight = 50
    MASK_Offset_TopLeft = 50
    MASK_Offset_TopRight = 50
    MASK_Init_Thickness = 300
    ignore_mask_color = 255
    


    ## Base image info and Masking parameter setup
    ysize = image.shape[0]
    xsize = image.shape[1]
    MASK_Offset_Horizon = int(ysize / 2 + MASK_Offset_Horizon_Correction)
    MASK_Offset_BottomHorizon = ysize  
    
    ## Create four sided polygon array for targetted image extraction
    Left_Line_Mask = np.array([[(MASK_Offset_BotLeft,MASK_Offset_BottomHorizon),(xsize/2-MASK_Offset_TopLeft,MASK_Offset_Horizon),
                        (xsize/2+MASK_Offset_TopRight,MASK_Offset_Horizon),(MASK_Offset_BotLeft+MASK_Init_Thickness,MASK_Offset_BottomHorizon)]],
                        dtype=np.int32)
    Right_Line_Mask = np.array([[(xsize-MASK_Offset_BotRight-MASK_Init_Thickness,MASK_Offset_BottomHorizon),(xsize/2-MASK_Offset_TopLeft,MASK_Offset_Horizon),
                        (xsize/2+MASK_Offset_TopRight,MASK_Offset_Horizon),(xsize - MASK_Offset_BotRight,MASK_Offset_BottomHorizon)]],
                        dtype=np.int32)

def Update_Mask():
    """
    This function updates the masking. This function shows my limitations with Python.
    There must have been MANY easier/quicker/cleaner ways to implement this, but this is all I could come up with
    
    """
    ## Globals
    global Left_Line_Mask
    global Right_Line_Mask

    
    MASK_Line_Thickness_Half = 10
    numbers = [ int(x) for x in Left_Line_Info ]
    placeholder = (numbers[0]-MASK_Line_Thickness_Half,numbers[1],
                    numbers[0]+MASK_Line_Thickness_Half,numbers[1],
                    numbers[2]-MASK_Line_Thickness_Half,numbers[3],
                    numbers[2]+MASK_Line_Thickness_Half,numbers[3])
    
    Left_Line_Mask = np.array([[(placeholder[0],placeholder[1]),(placeholder[2],placeholder[3]),
                               (placeholder[4],placeholder[5]),(placeholder[6],placeholder[7])]],
                              dtype=np.int32)
    
    numbers = [ int(x) for x in Right_Line_Info ]
    placeholder = (numbers[0]-MASK_Line_Thickness_Half,numbers[1],
                numbers[0]+MASK_Line_Thickness_Half,numbers[1],
                numbers[2]-MASK_Line_Thickness_Half,numbers[3],
                numbers[2]+MASK_Line_Thickness_Half,numbers[3])
    
    Right_Line_Mask = np.array([[(placeholder[0],placeholder[1]),(placeholder[2],placeholder[3]),
                               (placeholder[4],placeholder[5]),(placeholder[6],placeholder[7])]],
                              dtype=np.int32)

def Init():
    """
    Initialization of globals. Not sure if needed, maybe just best practices to find a place with all globals?
    
    
    """
    # These are all used for rudimentary learning of lane line placement
    global MASK_Offset_BottomHorizon
    global MASK_Offset_Horizon
    global Left_Line_Filter
    global Right_Line_Filter
    global Left_Line_Info
    global Right_Line_Info
    global Left_Line_Mask
    global Right_Line_Mask
    global Count

    print("INIT PROCESSED")

    ## Global End

    ## Init ##
    Left_Line_Filter = (-0.7,100) #Initial values just to have a base for slopes
    Right_Line_Filter = (0.7,100) #Initial values just to have a base for slopes

    Left_Line_Info = []
    Right_Line_Info = []
    

    Left_Line_Mask = []
    Right_Line_Mask = []
    
def Line_Averaging(Right_Line,Left_Line,Count):
    """
    As stated above, this function disappointed me. Lots of potential here for improvement
    More advanced averaging can and should be performed
    """
    ## Globals
    global Right_Line_Filter
    global Left_Line_Filter
    
    
    ## If statement for filtering
    if Count <= 6: #Below a threshold, no filtering to allow stabilization
        Right_Line_Filter = Right_Line
        Left_Line_Filter = Left_Line
    ## These next failure checks were for the challenge video and were not needed for the main 2 videos
    ## In effect, if there is a failure with the image to detect a realistic line, the filtering is skipped
    ## meaning previous values are maintained
    elif (np.isinf(Right_Line).any()):
        print("RFail")
    elif(np.isinf(Left_Line).any()):
        print("LFail")
    elif(not np.any(Right_Line)):
        print("Rzero")
    elif(not np.any(Left_Line)):
        print("Lzero")
    else:        
        Right_Line_Filter = (statistics.mean([Right_Line[0],Right_Line[0],Right_Line_Filter[0]]),statistics.mean([Right_Line[1],Right_Line[1],Right_Line_Filter[1]]))
        Left_Line_Filter =  (statistics.mean([Left_Line[0],Left_Line[0],Left_Line_Filter[0]]),statistics.mean([Left_Line[1],Left_Line[1],Left_Line_Filter[1]]))

    return



In [4]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    global MASK_Offset_BottomHorizon
    global MASK_Offset_Horizon
    global Count
    
    ## CONSTANTS START
    # Gaussian
    kernel_size = 5 #Result from homework lesson
    # Canny Edge
    low_threshold = 25
    high_threshold = 75
    # Hough
    rho = 2
    theta = np.pi/180
    threshold = 15
    min_line_length = 50
    max_line_gap = 40
    ## CONSTANTS END
    ## Count increment
    
    Count += 1
    
    ## GRAYSCALE
    ## Utilize OpenCV for grayscale conversion
    image_gray = grayscale(image)

    ## GAUSSIAN SMOOTHING
    ## Utilize OpenCV for gaussian smoothing
    blur_gray = gaussian_blur(image_gray,kernel_size)

    ## CANNY EDGE
    ## Utilize OpenCV for Canny Edge Detection
    edges = canny(blur_gray,low_threshold,high_threshold)
    
    ## MASKING
    if Count <= 1:
        Init()
        Init_Mask(edges)
    else:
        Update_Mask()
   
        
    ## Utilize numpy for init of mask of zero's for targetted image extraction
    mask1 = np.zeros_like(edges)
    mask2 = np.zeros_like(edges)
     
    cv2.fillPoly(mask1,Left_Line_Mask,255)
    cv2.fillPoly(mask2,Right_Line_Mask,255)
    mask3 = cv2.bitwise_or(mask1,mask2)
    masked_edges = cv2.bitwise_and(edges,mask3)
    line_image = np.copy(image)*0

    hough_out = hough_lines(masked_edges,rho,theta,threshold,min_line_length,max_line_gap,Count)


    weighted_out = weighted_img (hough_out,image)
   
    
    return weighted_out

Let's try the one with the solid white lane on the right first ...

In [5]:
global Count

Count = 0

white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

INIT PROCESSED
[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 221/222 [00:04<00:00, 48.72it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 4.98 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [7]:
global Count

Count = 0

yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

INIT PROCESSED
[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 681/682 [00:15<00:00, 44.60it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 15.8 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

## USER INPUT REFLECTIONS
Overall, I was happy with the project. I will admit I have limited experience with Python and its syntax, so this was a great opportunity for me to grow in this. I am very proud of the pipeline I created and the results it generated (although, not too happy about my failure on the challenge video). 

I feel that the low hanging fruit for my pipeline would be my averaging. With my implementation, I am effectively only applying a weighted filter with 2 options, current line and previous line. This is not enough. I'm sure there are complex averaging routines I could use, and I'm excited to learn more about them.

I am happy with my masking routine, and the next step there would be continuing to refine this algorithm and adding a function that allows my horizons (top and bottom limits) to change as the road condition changes. 

It should be noted, though, that this P1 notebook was an update to a previous version that actually worked BETTER for the challenge video. I did not want to revert because I think that some minor tweaks to the new functions I have implemented in this version will make this a very strong algorithm. In my final submission, I did include the 'better' challenge output labelled 'extra_BetterVersion.mp4'. The version that generated this mp4 can be found @https://github.com/BHunt11/CarND-LaneLines-P1/commit/1e923731c908f509e78dab3dff25e24cf0c1c891




## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [9]:
global Count

Count = 0

challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

INIT PROCESSED
[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


 41%|███████████████████████████████████████████████▏                                                                    | 102/251 [00:03<00:05, 26.95it/s]

Lzero


 42%|████████████████████████████████████████████████▌                                                                   | 105/251 [00:03<00:05, 27.18it/s]

Lzero
Lzero


...

Lzero


 81%|██████████████████████████████████████████████████████████████████████████████████████████████▎                     | 204/251 [00:08<00:02, 20.37it/s]

Lzero


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:10<00:00, 24.92it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 10.9 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))